In [2]:
import pandas as pd
from difflib import get_close_matches
import unicodedata


# Données GOLD

In [4]:
data_raw = pd.read_excel('FICHIER GOLD 2025.xlsx', sheet_name='BDD')
data_raw.head()

,Nom,Prenom,Mail,Telephone,Nom de l'entreprise,Addresse de l'entreprise,Code Postal de l'entreprise,Ville de l'entreprise,Message,Envoyé à,Nom du Magasin,Magasin Code Postal,Magasin Addresse,Nom de l'Association,Id de l'Association
0,BODSON-CASSAN,Blanche,blanche.cassan@laposte.net,681098969,BODSON,ARCUEIL,94117,ARCUEIL,TEST,13/09/2025 11h00,FRANPRIX,75001,16 rue Bertin Poirée,NaN,asso-A00007
1,Mallet,Sabine,mallet.sabine@gmail.com,603543624,Inetum,5 rue Touzet Gaillard,93400,Saint-Ouen,Bonjour \nJe suis disponible le vendredi 28 no...,17/09/2025 16h57,MONOPRIX,94120,96 rue Dalayrac CTM1134,Secours Catholique Fontenay sous Bois,asso-9413
2,Dequecker,Charlotte,dequeckercharlotte@gmail.com,766788916,NaN,NaN,NaN,NaN,Bonjour j’ai déjà participé à la collecte alim...,17/09/2025 22h16,MONOPRIX,75014,8 avenue du Général Leclerc,Aide Alimentaire Alésia,asso-1403
3,Verclytte,Daphne,verclytte.daphne@gmail.com,767213435,NaN,NaN,NaN,NaN,"Bonjour, \nEtudiante en relations internationa...",18/09/2025 15h59,MONOPRIX,75014,31 rue du Départ,Montparnasse Rencontres,asso-0606
4,Sellenet,Marie,msellenet@gmail.com,614114572,Groupe ADP,1 rue de France,NaN,Tremblay en France,"Bonjour, ayant déjà participé l'année dernière...",19/09/2025 16h18,MONOPRIX,92120,86 rue de la République,NaN,asso-R6480


In [5]:
data_raw.columns.tolist()

['Nom',
 'Prenom',
 'Mail',
 'Telephone',
 "Nom de l'entreprise",
 "Addresse de l'entreprise",
 "Code Postal de l'entreprise",
 "Ville de l'entreprise",
 'Message',
 'Envoyé à',
 'Nom du Magasin',
 'Magasin Code Postal',
 'Magasin Addresse',
 "Nom de l'Association",
 "Id de l'Association"]

In [6]:
data_raw.shape

(1103, 15)

In [7]:
data_raw_duplicates = data_raw[data_raw.duplicated(subset=['Nom',
 'Prenom',
 'Mail'], keep='first')]
data_raw_duplicates.shape

(304, 15)

In [8]:
data_raw_duplicates.head()

,Nom,Prenom,Mail,Telephone,Nom de l'entreprise,Addresse de l'entreprise,Code Postal de l'entreprise,Ville de l'entreprise,Message,Envoyé à,Nom du Magasin,Magasin Code Postal,Magasin Addresse,Nom de l'Association,Id de l'Association
6,Mallet,Sabine,mallet.sabine@gmail.com,603543624,INETUM,5 rue Touzet Gaillard,93400,Saint-Ouen,"Bonjour \n\nDans le cadre RSE, mon entreprise ...",22/09/2025 13h53,MONOPRIX,94300,27 rue de l'Eglise,NaN,asso-L2600
8,Grillier,Bertille,bertillegrillier@gmail.com,777700436,NaN,NaN,NaN,NaN,NaN,23/09/2025 09h43,MONOPRIX,75017,159 rue de Courcelles,NaN,asso-R6445
9,Grillier,Bertille,bertillegrillier@gmail.com,777700436,NaN,NaN,NaN,NaN,NaN,23/09/2025 10h48,FRANPRIX,75015,65-77 rue Falguière,CRF UL Paris 15ème L'Etape,asso-1512
10,Grillier,Bertille,bertillegrillier@gmail.com,777700436,NaN,NaN,NaN,NaN,NaN,23/09/2025 10h48,FRANPRIX,75015,65-77 rue Falguière,CRF UL Paris 15ème L'Etape,asso-1512
12,de sury,laure,laure.desurydaspremont@student-cs.fr,614069238,NaN,NaN,NaN,NaN,NaN,23/09/2025 10h50,FRANPRIX,75015,65-77 rue Falguière,CRF UL Paris 15ème L'Etape,asso-1512


# Entreprise Komeet

In [9]:
komeet_companies = pd.read_csv('Liste des entreprises KOMEET  - Liste principale.csv')
komeet_companies.shape

(529, 1)

In [10]:
komeet_companies.columns.tolist()

["Nom de l'entreprise"]

# Personnes de Komeet & GOLD

In [11]:
# Filtrer en tolerant les variations orthographiques des entreprises
def normalize_company(name: str) -> str:
    if pd.isna(name):
        return ""
    normalized = unicodedata.normalize('NFKD', str(name).lower())
    return ' '.join(normalized.split())

entreprises_komeet = (
    komeet_companies["Nom de l'entreprise"]
    .dropna()
    .map(normalize_company)
)

entreprises_komeet = [
    entreprise for entreprise in entreprises_komeet.unique()
    if entreprise
]

def find_match(entreprise: str, cutoff: float = 0.75):
    normalized = normalize_company(entreprise)
    if not normalized:
        return None
    matches = get_close_matches(normalized, entreprises_komeet, n=1, cutoff=cutoff)
    if not matches:
        return None
    return matches[0]

matches = data_raw["Nom de l'entreprise"].apply(find_match)
data_komeet = data_raw[matches.notna()].copy()
data_komeet['Entreprise_komeet_match'] = matches.loc[data_komeet.index]
data_komeet.head()


,Nom,Prenom,Mail,Telephone,Nom de l'entreprise,Addresse de l'entreprise,Code Postal de l'entreprise,Ville de l'entreprise,Message,Envoyé à,Nom du Magasin,Magasin Code Postal,Magasin Addresse,Nom de l'Association,Id de l'Association,Entreprise_komeet_match
4,Sellenet,Marie,msellenet@gmail.com,614114572,Groupe ADP,1 rue de France,NaN,Tremblay en France,"Bonjour, ayant déjà participé l'année dernière...",19/09/2025 16h18,MONOPRIX,92120,86 rue de la République,NaN,asso-R6480,groupe adp
14,ESSIRARD,Léa,lessirard@expanscience.com,781862259,Expanscience,"Tour First, 1 Pl. des Saisons",92400,Courbevoie,"Bonjour, \nJe souhaiterai participer avec mon ...",24/09/2025 12h37,MONOPRIX,92400,12 rue de l'Abreuvoir CTM2341,NaN,asso-A1040,expanscience
15,ESSIRARD,Léa,lessirard@expanscience.com,781862259,Expanscience,"Tour First, 1 Pl. des Saisons",92400,Courbevoie,"Bonjour, \nJe souhaiterai participer avec mon ...",24/09/2025 12h37,MONOPRIX,92400,12 rue de l'Abreuvoir CTM2341,NaN,asso-A1040,expanscience
16,Millot,benoit,benoit.millot@sanofi.com,648828664,Sanofi,82 avenue Raspail,94250,GENTILLY,NaN,24/09/2025 14h38,MONOPRIX,92270,ZAC Bruyères R Guynemer,Société SVP Conseil dpt 92,asso-9212,sanofi
17,CLAIR,Samira,samira.clair@bnpparibas.com,666962225,BNP PARIBAS,16 boulevard des italiens,75009,paris,NaN,25/09/2025 14h22,MONOPRIX,75001,21 avenue de l'Opéra,Soupe Saint Eustache,asso-0103,bnp paribas france


## Personnes de Komeet à envoyer

In [12]:
# Correction : on utilise [[]] pour sélectionner plusieurs colonnes
data_komeet = data_komeet[['Nom', 'Prenom', 'Mail', "Nom de l'entreprise", 'Nom de l\'Association']]

# Afficher les premières lignes du DataFrame résultant
data_komeet.head()

,Nom,Prenom,Mail,Nom de l'entreprise,Nom de l'Association
4,Sellenet,Marie,msellenet@gmail.com,Groupe ADP,NaN
14,ESSIRARD,Léa,lessirard@expanscience.com,Expanscience,NaN
15,ESSIRARD,Léa,lessirard@expanscience.com,Expanscience,NaN
16,Millot,benoit,benoit.millot@sanofi.com,Sanofi,Société SVP Conseil dpt 92
17,CLAIR,Samira,samira.clair@bnpparibas.com,BNP PARIBAS,Soupe Saint Eustache


In [13]:
data_komeet.shape

(424, 5)

## Doublons de personnes de Komeet à envoyer

In [15]:
data_komeet_duplicates = data_komeet[data_komeet.duplicated(subset=['Nom','Prenom','Mail'], keep='first')]
data_komeet_duplicates.shape

(90, 5)

In [16]:
data_komeet_duplicates.head(10)

,Nom,Prenom,Mail,Nom de l'entreprise,Nom de l'Association
15,ESSIRARD,Léa,lessirard@expanscience.com,Expanscience,NaN
31,RENAULT,Hugues,hugues.renault@bnpparibas.com,BNP PARIBAS,Thorigny Solidaire
35,BERTRAND,Alexandra,alexandra.bertrand@bnpparibas.com,bnp paribas,Conf SVP Maisons-Alfort
52,Millot,benoit,benoit.millot@sanofi.com,SANOFI,NaN
86,Pan,Yiru,yiru.pab@bnpparibas.com,BNP Paribas,NaN
87,Pan,Yiru,yiru.pab@bnpparibas.com,BNP Paribas,NaN
88,Gaudefroy,Véronique,veronique.gaudefroy@free.fr,sanofi,L'Etape Palaiseau
102,SZLAPKA,LEA,szlapka.lea@gmail.com,CARREFOUR,Aide Alimentaire Alésia
112,JENNAH,Manal,manal.jennah@kellanova.com,KELLANOVA,CRF UL Paris 12ème Espace Solidaire
114,Hamati,Nicolas,nicolas.hamati@kellanova.com,Kellanova,Partage 5/6/7 Saint Thomas d'Aquin


# Liste des personnes Komeet avec le champ vide

In [19]:
# Identifier les contacts sans nom d'entreprise et se baser sur le domaine des mails
import re

mask_missing_company = (
    data_raw["Nom de l'entreprise"].isna()
    | data_raw["Nom de l'entreprise"].astype(str).str.strip().eq("")
)
contacts_sans_entreprise = data_raw.loc[mask_missing_company].copy()

contacts_sans_entreprise["email_domain"] = (
    contacts_sans_entreprise["Mail"]
    .str.extract(r"@([A-Za-z0-9.-]+)", expand=False)
    .str.lower()
)

def domain_to_key(domain: str) -> str:
    if not isinstance(domain, str) or not domain:
        return ""
    parts = domain.split(".")
    if len(parts) >= 2:
        candidate = parts[-2]
        if candidate in {"com", "org", "net", "co"} and len(parts) >= 3:
            candidate = parts[-3]
    else:
        candidate = parts[0]
    return re.sub(r"[^a-z0-9]", "", candidate)

contacts_sans_entreprise["email_key"] = contacts_sans_entreprise["email_domain"].map(domain_to_key)


def company_to_key(name: str) -> str:
    normalized = normalize_company(name)
    if not normalized:
        return ""
    return re.sub(r"[^a-z0-9]", "", normalized)

entreprises_komeet_lookup = (
    komeet_companies["Nom de l'entreprise"]
    .dropna()
    .to_frame(name="Nom de l'entreprise")
)

entreprises_komeet_lookup["company_key"] = entreprises_komeet_lookup["Nom de l'entreprise"].apply(company_to_key)
entreprises_komeet_lookup = (
    entreprises_komeet_lookup[entreprises_komeet_lookup["company_key"].ne("")]
    .drop_duplicates(subset="company_key")
    .set_index("company_key")["Nom de l'entreprise"]
)

contacts_sans_entreprise["Entreprise_komeet_match_mail"] = contacts_sans_entreprise["email_key"].map(entreprises_komeet_lookup)

data_mail_match = contacts_sans_entreprise[contacts_sans_entreprise["Entreprise_komeet_match_mail"].notna()].copy()
data_mail_match = data_mail_match[['Nom', 'Prenom', 'Mail', "Nom de l'entreprise", 'Nom de l\'Association', 'Entreprise_komeet_match_mail']]
data_mail_match.head()


,Nom,Prenom,Mail,Nom de l'entreprise,Nom de l'Association,Entreprise_komeet_match_mail
42,Trompille,Emilie,emilie.trompille@laposte.net,NaN,Carrefour des Solidarités,La Poste
43,Trompille,Emilie,emilie.trompille@laposte.net,NaN,Carrefour des Solidarités,La Poste
135,TEZKRATT,JALILA,jalila.tezkratt@edenred.com,NaN,Secours Adventiste de Bagneux,Edenred
136,TEZKRATT,JALILA,jalila.tezkratt@edenred.com,NaN,Secours Adventiste de Bagneux,Edenred
137,TEZKRATT,JALILA,jalila.tezkratt@edenred.com,NaN,Secours Adventiste de Bagneux,Edenred


In [26]:
data_mail_match.shape

(27, 6)

# Total des personnes Komeet à envoyer

In [21]:
full_data_komeet = pd.concat([data_komeet, data_mail_match], ignore_index=True)
full_data_komeet.shape

(451, 6)

In [22]:
full_data_komeet.head()

,Nom,Prenom,Mail,Nom de l'entreprise,Nom de l'Association,Entreprise_komeet_match_mail
0,Sellenet,Marie,msellenet@gmail.com,Groupe ADP,NaN,NaN
1,ESSIRARD,Léa,lessirard@expanscience.com,Expanscience,NaN,NaN
2,ESSIRARD,Léa,lessirard@expanscience.com,Expanscience,NaN,NaN
3,Millot,benoit,benoit.millot@sanofi.com,Sanofi,Société SVP Conseil dpt 92,NaN
4,CLAIR,Samira,samira.clair@bnpparibas.com,BNP PARIBAS,Soupe Saint Eustache,NaN


In [23]:
full_data_komeet.to_csv('liste_personnes_komeet.csv', index=False)

In [25]:
full_data_komeet.columns.tolist()

['Nom',
 'Prenom',
 'Mail',
 "Nom de l'entreprise",
 "Nom de l'Association",
 'Entreprise_komeet_match_mail']